## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!

**imports**

In [131]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import nltk
nltk.download('wordnet')
nltk.download('stopwords')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours


import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import TweetTokenizer
import string
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from wordcloud import WordCloud

import PIL
from PIL import Image, ImageDraw, ImageFont

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\16047\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16047\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'wordcloud'

**Import DATA**

In [3]:
df = pd.read_csv("..\Data/judge_tweet_company.csv", encoding= 'unicode_escape')

In [4]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


**Data Cleaning**

In [5]:
df = df.rename(columns={'tweet_text': 'tweet', 'emotion_in_tweet_is_directed_at': 'product_or_company', 'is_there_an_emotion_directed_at_a_brand_or_product': 'emotion_detected'} )

In [6]:
df.head()

,tweet,product_or_company,emotion_detected
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [8]:
df['emotion_detected'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: emotion_detected, dtype: int64

In [70]:
emotion_dict={'No emotion toward brand or product': 'Neutral', 
              'Positive emotion': 'Positive',
              'Negative emotion': 'Negative', 
              'I can\t tell': 'Unknown'}

In [71]:
df['emotion_detected'] = df['emotion_detected'].map(emotion_dict)

In [72]:
df.head(10)

,tweet,product_or_company,emotion_detected,cleaned_text,company
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative,wesley g iphone hr tweeting rise_austin dead n...,iPhone
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive,jessedee know fludapp awesome ipadiphone app y...,iPad or iPhone App
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive,swonderlin wait ipad also sale sxsw,iPad
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative,sxsw hope year festival isnt crashy year iphon...,iPad or iPhone App
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive,sxtxstate great stuff fri sxsw marissa mayer g...,Google
5,@teachntech00 New iPad Apps For #SpeechTherapy...,ipad,Neutral,teachntech new ipad apps speechtherapy communi...,ipad
6,NaN,NaN,Neutral,,NaN
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive,sxsw starting ctia around corner googleio hop ...,Android
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive,beautifully smart simple idea rt madebymany th...,iPad or iPhone App
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive,counting day sxsw plus strong canadian dollar ...,Apple


In [73]:
df['emotion_detected'].value_counts()

Neutral     5389
Positive    2978
Negative     570
Name: emotion_detected, dtype: int64

**Need to clean text to further clean rest of data**

In [47]:
def clean_text(tweet):
    if isinstance(tweet, str):  
        tweet = tweet.lower()
    
  
        tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)
    
  
        tweet = re.sub(r"[^\w\s]", "", tweet)
        tweet = re.sub(r"\d+", "", tweet)
    
   
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(tweet)
    
    
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
    
    
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
  
    
        clean_text = " ".join(tokens)
    
    
        return clean_text

        

In [48]:
df["cleaned_text"] = df["tweet"].apply(clean_text)

In [53]:
df["cleaned_text"] = df["cleaned_text"].fillna("")

In [49]:
df.head()

,tweet,product_or_company,emotion_detected,cleaned_text,company
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,wesley g iphone hr tweeting rise_austin dead n...,iPhone
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,jessedee know fludapp awesome ipadiphone app y...,iPad or iPhone App
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,swonderlin wait ipad also sale sxsw,iPad
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,sxsw hope year festival isnt crashy year iphon...,iPad or iPhone App
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,sxtxstate great stuff fri sxsw marissa mayer g...,Google


In [50]:
df.isna().sum()

tweet                   1
product_or_company    765
emotion_detected        0
cleaned_text            1
company               765
dtype: int64

In [16]:
keywords = ['google', 'apple', 'ipad', 'android', 'iphone']

for index, row in df.iterrows():
    cleaned_text = row['cleaned_text']
    if pd.isna(row['product_or_company']) and isinstance(cleaned_text, str):
        for keyword in keywords:
            if keyword in cleaned_text.lower():
                df.at[index, 'product_or_company'] = keyword
                break


In [17]:
df.isna().sum()

tweet                   1
product_or_company    765
emotion_detected        0
cleaned_text            1
dtype: int64

In [18]:
df.head()

,tweet,product_or_company,emotion_detected,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,wesley g iphone hr tweeting rise_austin dead n...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,jessedee know fludapp awesome ipadiphone app y...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,swonderlin wait ipad also sale sxsw
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,sxsw hope year festival isnt crashy year iphon...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,sxtxstate great stuff fri sxsw marissa mayer g...


In [167]:
df['company_apple'] = df['product_or_company'].replace(['iPhone', 
                                                  'iPad', 
                                                  'apple', 
                                                  'iPad or iPhone App',
                                                  'iphone', 
                                                  'ipad', 
                                                  'Other Apple product or service'], 'Apple')

In [169]:
df['company_google'] = df['product_or_company'].replace(['android', 
                                                  'Android', 
                                                  'google',
                                                  'Other Google product or service',
                                                  'Android App'], 'Google')

In [170]:
df['company_google'].value_counts()

Google                            2945
apple                             1195
ipad                              1070
iPad                               946
iphone                             709
Apple                              661
iPad or iPhone App                 470
iPhone                             297
Other Apple product or service      35
Name: company_google, dtype: int64

In [168]:
df['company_apple'].value_counts()

Apple                              5383
google                             1737
Google                              430
android                             326
Other Google product or service     293
Android App                          81
Android                              78
Name: company_apple, dtype: int64

In [171]:
df['company'] = df['company_apple'] + df['company_google']

In [172]:
df['company'].drop_duplicates()

0                                AppleiPhone
1                    AppleiPad or iPhone App
2                                  AppleiPad
4                               GoogleGoogle
5                                  Appleipad
6                                        NaN
7                              AndroidGoogle
9                                 AppleApple
11                         Android AppGoogle
13     Other Google product or serviceGoogle
32                             androidGoogle
35                              googleGoogle
42                               Appleiphone
60                                Appleapple
703      AppleOther Apple product or service
Name: company, dtype: object

In [173]:
df.head()

,tweet,product_or_company,emotion_detected,cleaned_text,company,company_apple,company_google
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative,wesley g iphone hr tweeting rise_austin dead n...,AppleiPhone,Apple,iPhone
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive,jessedee know fludapp awesome ipadiphone app y...,AppleiPad or iPhone App,Apple,iPad or iPhone App
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive,swonderlin wait ipad also sale sxsw,AppleiPad,Apple,iPad
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative,sxsw hope year festival isnt crashy year iphon...,AppleiPad or iPhone App,Apple,iPad or iPhone App
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive,sxtxstate great stuff fri sxsw marissa mayer g...,GoogleGoogle,Google,Google


**Time to start modeling**

In [51]:
sentences = df['cleaned_text']
labels = df['emotion_detected']

In [63]:
tdifvec = TfidfVectorizer()
X = tdifvec.fit_transform(sentences)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


#smote_enn = SMOTEENN(random_state=42, 
                     #smote=SMOTE(random_state=42), 
                     #enn=EditedNearestNeighbours(),
                     #sampling_strategy='auto')

#X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)


knn = KNeighborsClassifier(n_neighbors=2, 
                           weights ='distance',
                           )

knn.fit(X_train_resampled, y_train_resampled)


y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [124]:
print(accuracy)

0.6327652556349642


In [126]:
mnb = MultinomialNB()

In [128]:
mnb.fit(X_train_resampled, y_train_resampled)

y_pred = mnb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.02      0.06      0.03        32
           1       0.26      0.52      0.35       115
           2       0.76      0.54      0.63      1083
           3       0.51      0.61      0.55       589

    accuracy                           0.55      1819
   macro avg       0.39      0.43      0.39      1819
weighted avg       0.63      0.55      0.58      1819



In [62]:
#pipeline = Pipeline([
  #  ('preprocess', TfidfVectorizer(preprocessor=preprocess_tweet)),
  #  ('classifier', MultinomialNB())
#])


#X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)


#pipeline.fit(X_train, y_train)


#y_pred = pipeline.predict(X_test)


#print(classification_report(y_test, y_pred))

                                    precision    recall  f1-score   support

                      I can't tell       0.00      0.00      0.00        29
                  Negative emotion       0.50      0.01      0.02        96
No emotion toward brand or product       0.63      0.93      0.75       939
                  Positive emotion       0.70      0.32      0.44       597

                          accuracy                           0.64      1661
                         macro avg       0.46      0.32      0.30      1661
                      weighted avg       0.64      0.64      0.58      1661



C:\Users\16047\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [139]:
df['company'].value_counts()

google                             2063
apple                              1195
ipad                               1070
iPad                                946
iphone                              709
Apple                               661
iPad or iPhone App                  470
Google                              430
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: company, dtype: int64

In [138]:
df['product_or_company'].value_counts()

google                             1737
apple                              1195
ipad                               1070
iPad                                946
iphone                              709
Apple                               661
iPad or iPhone App                  470
Google                              430
android                             326
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: product_or_company, dtype: int64

In [133]:
freq = FreqDist(df['cleaned_text'])
freq.most_common(20)

[('rt mention google launch major new social network called circle possibly today link sxsw',
  13),
 ('rt mention marissa mayer google connect digital amp physical world mobile link sxsw',
  10),
 ('google launch major new social network called circle possibly today link sxsw',
  8),
 ('rt mention rumor apple opening temporary store downtown austin sxsw ipad launch link',
  8),
 ('win free ipad webdoccom sxsw rt', 7),
 ('rt mention rt mention rumor apple opening temporary store downtown austin sxsw ipad launch link',
  6),
 ('¼ _ link ã_ edchat musedchat sxsw sxswi classical newtwitter', 4),
 ('sxsw apple school marketing expert link', 4),
 ('marissa mayer google connect digital amp physical world mobile link sxsw',
  4),
 ('rt mention rt mention google launch major new social network called circle possibly today link sxsw',
  4),
 ('even begin apple win sxsw link', 3),
 ('google launch major new social network called circle link sxsw', 3),
 ('google launch major new social network ca

In [134]:
stop_lst = stopwords.words('english')
stop_lst += list(string.punctuation)
stop_lst += ['...','#sxsw', '’', '#sxswi', 'sxsw', "''", 'link','rt']

In [136]:
def remove_stopwords(tokens, stop_list = stop_lst):
    
   
    
    tokens_stopped = [word.encode('ascii','ignore').decode() 
                      for word in tokens 
                      if (word not in stop_list) & 
                      (word.startswith('http') == False)]
    

    return tokens_stopped

stop_lst_no_brands_products = stop_lst + ['ipad', 'apple', 'google', 'iphone', '#apple', 
                               '#google', '#ipad', '#iphone', 'android']

tokens_apple_pos_nostopwords = remove_stopwords(df['cleaned_text'], stop_lst_no_brands_products)

In [137]:
freq = FreqDist(tokens_apple_pos_nostopwords)
freq.most_common(40)

[('rt mention google launch major new social network called circle possibly today link sxsw',
  15),
 ('rt mention marissa mayer google connect digital amp physical world mobile link sxsw',
  10),
 ('google launch major new social network called circle possibly today link sxsw',
  9),
 ('rt mention rumor apple opening temporary store downtown austin sxsw ipad launch link',
  8),
 ('win free ipad webdoccom sxsw rt', 7),
 ('rt mention rt mention rumor apple opening temporary store downtown austin sxsw ipad launch link',
  6),
 ('rt mention rt mention google launch major new social network called circle possibly today link sxsw',
  5),
 (' _ link _ edchat musedchat sxsw sxswi classical newtwitter', 4),
 ('sxsw apple school marketing expert link', 4),
 ('mention new ubersocial iphone app store includes uberguide sxsw sponsored mashable link',
  4),
 ('marissa mayer google connect digital amp physical world mobile link sxsw',
  4),
 ('even begin apple win sxsw link', 3),
 ('google launch ma